In [320]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sea

import warnings
warnings.filterwarnings('ignore')

In [321]:
data = pd.read_csv('/Users/ankusmanish/Desktop/Training/Datasets/Week 6 data/bank.csv', sep = ';')

In [322]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [323]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
age          4521 non-null int64
job          4521 non-null object
marital      4521 non-null object
education    4521 non-null object
default      4521 non-null object
balance      4521 non-null int64
housing      4521 non-null object
loan         4521 non-null object
contact      4521 non-null object
day          4521 non-null int64
month        4521 non-null object
duration     4521 non-null int64
campaign     4521 non-null int64
pdays        4521 non-null int64
previous     4521 non-null int64
poutcome     4521 non-null object
y            4521 non-null object
dtypes: int64(7), object(10)
memory usage: 600.5+ KB


In [324]:
lis = []
cols = data.columns
for i in cols:
    if data[i].dtype == 'O':
        lis.append(i)

In [325]:
#The list of columns that has categorical values
print(lis)

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']


In [326]:
for i in lis:
    print(i.upper())
    print(data[i].value_counts())
    print('-'*100)
    print('-'*100)

JOB
management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
Name: job, dtype: int64
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
MARITAL
married     2797
single      1196
divorced     528
Name: marital, dtype: int64
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
EDUCATION
secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64
----------------------------------------------------------------------------------------------------
----------------

In [327]:
outcome = data['y']

In [328]:
data.drop(['job','default','housing','loan','month','poutcome','contact','day','marital','education','y'], axis = 1, inplace = True)

In [329]:
data.head()

,age,balance,duration,campaign,pdays,previous
0,30,1787,79,1,-1,0
1,33,4789,220,1,339,4
2,35,1350,185,1,330,1
3,30,1476,199,4,-1,0
4,59,0,226,1,-1,0


In [330]:
outcome = pd.DataFrame(outcome, columns = ['y'])

In [331]:
outcome.head()

,y
0,no
1,no
2,no
3,no
4,no


In [332]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

In [333]:
dat = lb.fit_transform(outcome)

In [334]:
outcome = pd.DataFrame(dat, columns = ['y'])

In [335]:
outcome['y'].value_counts()

0    4000
1     521
Name: y, dtype: int64

In [336]:
mod_data = pd.concat([data, outcome], axis = 1)

In [337]:
mod_data.head()

,age,balance,duration,campaign,pdays,previous,y
0,30,1787,79,1,-1,0,0
1,33,4789,220,1,339,4,0
2,35,1350,185,1,330,1,0
3,30,1476,199,4,-1,0,0
4,59,0,226,1,-1,0,0


In [338]:
mod_data.columns

Index(['age', 'balance', 'duration', 'campaign', 'pdays', 'previous', 'y'], dtype='object')

In [339]:
data = pd.DataFrame(mod_data, columns = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous', 'y'])

In [340]:
data.head()

,age,balance,duration,campaign,pdays,previous,y
0,30,1787,79,1,-1,0,0
1,33,4789,220,1,339,4,0
2,35,1350,185,1,330,1,0
3,30,1476,199,4,-1,0,0
4,59,0,226,1,-1,0,0


In [415]:
len(data[data['pdays'] < 0])

3705

In [347]:
train = data.iloc[:4000,:]
test = data.iloc[4000:,:]

In [345]:
train.head()

,age,balance,duration,campaign,pdays,previous,y
0,30,1787,79,1,-1,0,0
1,33,4789,220,1,339,4,0
2,35,1350,185,1,330,1,0
3,30,1476,199,4,-1,0,0
4,59,0,226,1,-1,0,0


In [346]:
test.tail()

,age,balance,duration,campaign,pdays,previous,y
4516,33,-333,329,5,-1,0,0
4517,57,-3313,153,1,-1,0,0
4518,57,295,151,11,-1,0,0
4519,28,1137,129,4,211,3,0
4520,44,1136,345,2,249,7,0


In [416]:
def E_Distance(x1, x2, length):
    distance = 0
    for x in range(length):
        distance += np.square(x1[x] - x2[x])
    return np.sqrt(distance)

In [431]:
# making function for defining K-NN model
def knn(train, test, k):
    distances = {}
    length = test.shape[1]
    for x in range(len(train)):
        dist = E_Distance(test, train.values[x], length)
        distances[x] = dist[0]
    sortdist = sorted(distances.items(), key = lambda x : x[1])
    
    
    neighbors = []
    for x in range(k):
        neighbors.append(sortdist[x][0])
    Count = {} 
    for x in range(len(neighbors)):
        response = train.iloc[neighbors[x]][-1]
        if response in Count:
            Count[response] += 1
        else:
            Count[response] = 1
    sortcount = sorted(Count.items(), key = lambda x : x[1], reverse=True)
    return (sortcount[0][0], neighbors)

In [432]:
# neighbors = [3981, 2019, 1071]
# Count = {}  # to get most frequent class of rows
# for x in range(len(neighbors)):
#     response = train.iloc[neighbors[x]][-1]
#     if response in Count:
#         Count[response] += 1
#     else:
#         Count[response] = 1
# sortcount = sorted(Count.items(), key=operator.itemgetter(1), reverse=True)

In [404]:
# sortcount

[(0, 3)]

In [442]:
# E_Distance(train.iloc[3981].values, b, 6)

0    16.248077
Name: 0, dtype: float64

In [439]:
# train.iloc[314][-1]

1

In [393]:
# distances = {}
# for x in range(len(train)):
#     dist = E_Distance(b, train.iloc[x], 7)
#     distances[x] = dist[0]
# sortdist = sorted(distances.items(), key=operator.itemgetter(1))

In [394]:
# sortdist

[(3981, 16.24807680927192),
 (2019, 24.06241883103193),
 (1071, 31.906112267087632),
 (1405, 37.5632799419859),
 (2951, 39.54743986657038),
 (1226, 39.736632972611055),
 (776, 39.7994974842648),
 (380, 45.13313638558703),
 (1240, 57.113921245174545),
 (3434, 57.956880523368405),
 (3543, 70.49113419430843),
 (1323, 70.80254232723568),
 (883, 72.17340230306452),
 (2181, 73.08214556237385),
 (819, 73.71566997592845),
 (3487, 73.87151006985034),
 (2934, 83.18052656722004),
 (375, 83.27664738688752),
 (3976, 86.55634003352961),
 (2306, 91.65696918401787),
 (3060, 94.65199416811038),
 (187, 98.11218069128827),
 (3041, 98.2344135219425),
 (188, 98.28021163998376),
 (3669, 102.73266277090262),
 (2659, 106.59737332598773),
 (1109, 107.87492757819122),
 (913, 109.74971526158963),
 (3287, 111.51681487560519),
 (245, 113.41075786714416),
 (2000, 115.28226229563678),
 (2536, 132.1325092473461),
 (1426, 132.4235628579748),
 (1327, 132.58582126305964),
 (869, 132.7629466379833),
 (2069, 133.063894426

In [433]:
#making test data set
testSet = [[ 53,  26,  56,   1, 359,   1,   0]]
test = pd.DataFrame(testSet)

#assigning different values to k
k = 1
k1 = 3
k2 = 11


result, neigh = knn(train, test, k)
result1, neigh1 = knn(train, test, k1)
result2, neigh2 = knn(train, test, k2)



In [434]:
print(result)
print(neigh)
print(result1)
print(neigh1)
print(result2)
print(neigh2)

0
[3981]
0
[3981, 2019, 1071]
0
[3981, 2019, 1071, 1405, 2951, 1226, 776, 380, 1240, 3434, 3543]


In [360]:
# test.values[0]
# b = pd.DataFrame([[ 53,  26,  56,   1, 359,   1,   0]])

In [410]:
# result3, neigh4 = knn(train, b, 3)
# print(result3)
# print(neigh4)

0
[3981, 2019, 1071]
